## CLIP

In [20]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

In [2]:
clip_model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(clip_model_name)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [3]:
img = Image.open("/kaggle/input/cse344-a3/sample_image.jpg").convert("RGB")

texts = [
    "a person holding a dog",
    "a person with their pet dog",
    "a person and a furry friend",
    "a man holding a dog",
    "a man with his pet dog",
    "a man and a furry friend",
    "a dog with its owner",
    "a dog with its loving owner",
    "a loyal dog with its owner",
    "a loyal dog with its loving owner",
]

In [4]:
inputs = clip_processor(text=texts, images=img, return_tensors="pt", padding=True)
with torch.no_grad():
    outputs = clip_model(**inputs)
    logits_per_image = outputs.logits_per_image  # this is the image-text similarity score

print("Similarity Scores:")
for i, text in enumerate(texts):
    score = logits_per_image[0][i].item()
    print(f"'{text}': {score:.4f}")

Similarity Scores:
'a person holding a dog': 29.7640
'a person with their pet dog': 29.3991
'a person and a furry friend': 26.4187
'a man holding a dog': 30.5882
'a man with his pet dog': 29.9326
'a man and a furry friend': 28.3417
'a dog with its owner': 27.9548
'a dog with its loving owner': 26.8501
'a loyal dog with its owner': 25.7771
'a loyal dog with its loving owner': 25.1614


## CLIPS

In [5]:
!pip install open_clip_torch -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0

In [6]:
import torch.nn.functional as F
from open_clip import create_model_from_pretrained, get_tokenizer

In [7]:
clips_model, clips_preprocess = create_model_from_pretrained('hf-hub:UCSC-VLAA/ViT-L-14-CLIPS-224-Recap-DataComp-1B')
clips_tokenizer = get_tokenizer('hf-hub:UCSC-VLAA/ViT-L-14-CLIPS-224-Recap-DataComp-1B')

open_clip_pytorch_model.bin:   0%|          | 0.00/1.66G [00:00<?, ?B/s]

open_clip_config.json:   0%|          | 0.00/943 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [19]:
image = clips_preprocess(img).unsqueeze(0)
text = clips_tokenizer(texts, context_length=clips_model.context_length)

with torch.no_grad(), torch.amp.autocast('cuda'):
    image_features = clips_model.encode_image(image)
    text_features = clips_model.encode_text(text)
    image_features = F.normalize(image_features, dim=-1)
    text_features = F.normalize(text_features, dim=-1)
    similarity_scores = (100.0 * image_features @ text_features.T) #unnormalized similarity scores

print("Similarity Scores:")
for i, text in enumerate(texts):
    score = similarity_scores[0][i].item()
    print(f"'{text}': {score:.4f}")

Similarity Scores:
'a person holding a dog': 13.2046
'a person with their pet dog': 12.3108
'a person and a furry friend': 12.8810
'a man holding a dog': 16.3031
'a man with his pet dog': 15.4424
'a man and a furry friend': 15.2662
'a dog with its owner': 15.0931
'a dog with its loving owner': 15.3591
'a loyal dog with its owner': 14.7291
'a loyal dog with its loving owner': 15.2724
